In [ ]:
import os
    
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import seaborn as sns

from tqdm import tqdm
from scipy.spatial.distance import pdist, cdist, squareform
from sklearn.manifold import MDS

from sklearn.svm import LinearSVC
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler

RAND = 0
RESP = (50,220)
BASE = (-50,0)
ONSET = 50
RESP = slice(ONSET + RESP[0], ONSET + RESP[1])
BASE = slice(ONSET + BASE[0], ONSET + BASE[1])

In [ ]:
DATA_DIR = '../../datasets/NNN/'
CAT = 'face'
dat = pd.read_pickle(os.path.join(DATA_DIR, (f'{CAT}_roi_data.pkl')))
print(f'Unique {CAT} ROIs: {list(dat['roi'].unique())}')

# DIRECTORY FOR FIGURES
SAVE_DIR = './../../../buckets/manifold-dynamics/time-time'
if not os.path.exists(SAVE_DIR):
    os.makedirs(SAVE_DIR)

In [ ]:
def geo_rdm(dat, roi, mode='top', step=5, k_max=200, metric='correlation'):
    rng = np.random.default_rng(RAND)
    sig = dat[dat['p_value'] < 0.05]
    df = sig[sig['roi'] == roi]
    if len(df) == 0:
        raise ValueError(f"No data for ROI {ROI}")
    X = np.stack(df['img_psth'].to_numpy())          # (units, time, images)

    scores = np.nanmean(X[:, RESP, :], axis=(0,1)) - np.nanmean(X[:, BASE, :], axis=(0,1))
    order = np.argsort(scores)[::-1] if mode == 'top' else rng.permutation(scores.size)

    # ================= choose the image-set bins to calculate RDMs ========
    sizes = [k for k in range(step, min(k_max, X.shape[2]) + 1, step)]
    # =================== ramping step size ================================ 
    # sizes = [k for k in range(1, 2*step)] + [k for k in range(2*step, min(k_max, X.shape[2])+1, step)]
    
    rdvs = []
    for k in tqdm(sizes):
        idx = order[:k]
        Ximg = X[:, :, idx] # (units, time, images)
        Xrdv = np.array([pdist(Ximg[:, t, :].T, metric='correlation') for t in range(Ximg.shape[1])])
        R = squareform(pdist(Xrdv, metric=metric))   # (time, time)
        rdvs.append(R)
    return sizes, rdvs

def rdvs(dat, roi, mode='top', step=5, k_max=200, metric='correlation'):
    rng = np.random.default_rng(RAND)
    sig = dat[dat['p_value'] < 0.05]
    df = sig[sig['roi'] == roi]
    if len(df) == 0:
        raise ValueError(f"No data for ROI {ROI}")
    X = np.stack(df['img_psth'].to_numpy())          # (units, time, images)

    scores = np.nanmean(X[:, RESP, :], axis=(0,1)) - np.nanmean(X[:, BASE, :], axis=(0,1))
    order = np.argsort(scores)[::-1] if mode == 'top' else rng.permutation(scores.size)

    # ================= choose the image-set bins to calculate RDMs ========
    sizes = [k for k in range(step, min(k_max, X.shape[2]) + 1, step)]
    # =================== ramping step size ================================ 
    # sizes = [k for k in range(1, 2*step)] + [k for k in range(2*step, min(k_max, X.shape[2])+1, step)]
    
    rdvs = []
    for k in tqdm(sizes):
        idx = order[:k]
        Ximg = X[:, :, idx] # (units, time, images)
        Xrdv = np.array([pdist(Ximg[:, t, :].T, metric='correlation') for t in range(Ximg.shape[1])])
        rdvs.append(Xrdv)
    return sizes, rdvs

'''
THESE TWO FUNCTIONS MAKE RDMs USING A SINGEL SET OF IMAGES
 (AS OPPOSED TO THE TOP/SHUFFLED)
'''
def gain(dat, roi, img_sets, mode='top', step=5, k_max=200, metric='correlation'):
    sig = dat[dat['p_value'] < 0.05]
    df = sig[sig['roi'] == roi]
    if len(df) == 0:
        raise ValueError(f"No data for ROI {ROI}")
    X = np.stack(df['img_psth'].to_numpy())          # (units, time, images)

    rdvs = {}
    for k,v in img_sets.items():
        Ximg = X[:, :, v]
        Ximg = np.mean(Ximg, axis=0)
        R = squareform(pdist(Ximg, metric=metric))
        rdvs[k] = R
    return rdvs

def geom(dat, roi, img_sets, mode='top', step=5, k_max=200, metric='correlation'):
    sig = dat[dat['p_value'] < 0.05]
    df = sig[sig['roi'] == roi]
    if len(df) == 0:
        raise ValueError(f"No data for ROI {ROI}")
    X = np.stack(df['img_psth'].to_numpy())          # (units, time, images)

    rdvs = {}
    time_rdvs = {}
    for k,v in img_sets.items():
        Ximg = X[:, :, v]
        Xrdv = np.array([pdist(Ximg[:, t, :].T, metric='correlation') for t in range(Ximg.shape[1])])
        time_rdvs[k] = Xrdv
        R = squareform(pdist(Xrdv, metric=metric))
        rdvs[k] = R
    return rdvs, time_rdvs

In [ ]:
_roi = 'MF1_8_F' # MF1_8_F, Unknown_19_F, MF1_7_F, MF1_9_F

img_sets = {
    'all_images': np.arange(1000, 1072),
    "all_faces":  np.arange(1000, 1024),
    # 'monkey faces':  np.concatenate([np.arange(1000,1006), np.arange(1009,1016)]),
    # 'human faces': np.concatenate([np.arange(1006,1009), np.arange(1016,1025)]),
    "all_bodies": np.concatenate([
        np.arange(1025, 1031),
        np.arange(1042, 1048),
        np.arange(1049, 1061)
    ]),
    "all_objects": np.concatenate([
        np.arange(1024,1025),
        np.arange(1031, 1042),
        np.arange(1048,1049),
        np.arange(1061, 1072)
    ])
}

rdms, Xrdv = geom(dat, _roi, img_sets)

for k,v in rdms.items():
    fig,ax = plt.subplots(1,1, figsize=(2,2))
    sns.heatmap(v, vmin = 0, vmax=1, square = True, cbar=False, ax=ax)

    # ax.set_title(k)
    ax.set_title('')
    
    ax.set_xticks([])
    ax.set_yticks([])
    
    ax.set_xlabel('Time')
    ax.set_ylabel('Time')

    ymin, ymax = ax.get_ylim()
    ax.vlines(x=50, ymin=ymin, ymax=ymax, color='red', linestyle='--')
    ax.hlines(y=50, xmin=ymin, xmax=ymax, color='red', linestyle='--')
    
    out_path = os.path.join(SAVE_DIR, 'static', f'{_roi}_{k}.png')
    plt.savefig(out_path, dpi=300, transparent=True, bbox_inches='tight')
    plt.show()

In [ ]:
R = rdms['all_images']
diag_vals = np.diag(R)
off_vals  = R[~np.eye(R.shape[0], dtype=bool)]

plt.figure(figsize=(5,3))
sns.histplot(off_vals, bins=40, stat='probability', alpha=0.5, label="off-diagonal")
# sns.histplot(diag_vals, bins=40, stat='probability', alpha=0.5, label="on-diagonal")
plt.legend(); plt.xlabel("distance"); plt.ylabel("count")
plt.title("Diagonal vs off-diagonal distances")
plt.tight_layout(); plt.show()

# plt.figure(figsize=(5,3))
# plt.plot(diag_vals, label="diag")
# plt.plot(off_vals[::len(diag_vals)], alpha=0.4, label="sampled off-diag")
# plt.legend(); plt.tight_layout(); plt.show()

In [ ]:
### CATEGORY DECODING GENERALIZATION
gens_8 = {}
n_splits = 24 # or 24 for full leave-one-out

for _roi in tqdm(['MF1_8_F', 'Unknown_19_F', 'MF1_7_F', 'MF1_9_F']):
    # get data (units, time, images 1000:1072 → 72 imgs)
    X = np.stack(dat[(dat['roi']==_roi) & (dat['p_value']<0.05)]['img_psth'])
    X = X[:, :, 1000:]                    # (U, T, I)
    U, T, I = X.shape
    
    # reshape to (time, images, units)
    data = X.transpose(1, 2, 0).reshape(T, I, -1)
    
    # labels per image (0/1/2/3)
    labels = np.zeros(I, int)
    labels[img_sets['all_faces']   - 1000] = 1
    labels[img_sets['all_bodies']  - 1000] = 2
    labels[img_sets['all_objects'] - 1000] = 3
    
    # temporal generalization matrix with CV
    gen = np.zeros((T, T))
    skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=0)
    
    for train_idx, test_idx in skf.split(np.arange(I), labels):
        y_train = labels[train_idx]
        y_test  = labels[test_idx]
    
        for t_train in range(T):
            Xtr = data[t_train, train_idx, :]
            clf = LinearSVC(dual='auto')
            clf.fit(Xtr, y_train)
    
            for t_test in range(T):
                Xte = data[t_test, test_idx, :]
                dec = clf.predict(Xte)
                acc = (dec == y_test).mean()
                gen[t_train, t_test] += acc
    
    gen /= skf.get_n_splits()
    gens_8[_roi] = gen
    # np.fill_diagonal(gen, np.nan)

In [ ]:
ticks = np.arange(0, T, 100)

for _roi in ['Unknown_19_F', 'MF1_7_F', 'MF1_8_F', 'MF1_9_F']:
    fig, ax = plt.subplots(1,1, figsize=(2,2))
    gen = gens_8[_roi]
    sns.heatmap(gen, cmap="magma", square=True, 
                cbar=False, vmin=0.3, vmax=1, ax=ax)
    ax.set_xlabel("Test time")
    ax.set_ylabel("Train time")
    
    ax.set_xticks([])
    # ax.set_xticklabels(ticks)
    ax.set_yticks([])
    # ax.set_yticklabels(ticks)

    ymin, ymax = ax.get_ylim()
    ax.vlines(x=50, ymin=ymin, ymax=ymax, color='red', linestyle='--')
    ax.hlines(y=50, xmin=ymin, xmax=ymax, color='red', linestyle='--')

    out_path = os.path.join(SAVE_DIR, 'temporal-generalization', f'{_roi}_24_fold.png')
    plt.savefig(out_path, dpi=300, transparent=True, bbox_inches='tight')
    plt.show()

In [ ]:
# global vmin/vmax across all ROIs
vmin = np.nanmin([np.nanmin(g) for g in gens.values()])
vmax = np.nanmax([np.nanmax(g) for g in gens.values()])

# plot grid with shared colorbar
fig, axes = plt.subplots(1,4, figsize=(10, 3), sharey=True)
axes = axes.ravel()
ticks = np.arange(0, T, 100)
ROI_LIST = ['Unknown_19_F', 'MF1_7_F', 'MF1_8_F', 'MF1_9_F']

for ax, _roi in zip(axes, ROI_LIST):
    gen = gens[_roi]
    hm = sns.heatmap(gen, cmap="magma", square=True, cbar=False,
                     vmin=0.33, vmax=vmax, ax=ax)
    ax.set_title(_roi)
    ax.set_xlabel("Test time")
    ax.set_ylabel("Train time")
    ax.set_xticks(ticks)
    ax.set_xticklabels(ticks)
    ax.set_yticks(ticks)
    ax.set_yticklabels(ticks)

# single shared colorbar at bottom
cbar = fig.colorbar(axes[0].collections[0],
                    ax=axes,
                    orientation='vertical',
                    fraction=0.01,
                    pad=-0.15)
cbar.set_label("Decoding accuracy")

plt.tight_layout()
plt.show()

In [ ]:
# global vmin/vmax across all ROIs
vmin = np.nanmin([np.nanmin(g) for g in gens_8.values()])
vmax = np.nanmax([np.nanmax(g) for g in gens_8.values()])

# plot grid with shared colorbar
fig, axes = plt.subplots(1,4, figsize=(10, 3), sharey=True)
axes = axes.ravel()
ticks = np.arange(0, T, 100)
ROI_LIST = ['Unknown_19_F', 'MF1_7_F', 'MF1_8_F', 'MF1_9_F']

for ax, _roi in zip(axes, ROI_LIST):
    gen = gens_8[_roi]
    hm = sns.heatmap(gen, cmap="magma", square=True, cbar=False,
                     vmin=0.33, vmax=vmax, ax=ax)
    ax.set_title(_roi)
    ax.set_xlabel("Test time")
    ax.set_ylabel("Train time")
    ax.set_xticks(ticks, ticks)
    ax.set_yticks(ticks, ticks)


# single shared colorbar at bottom
cbar = fig.colorbar(axes[0].collections[0],
                    ax=axes,
                    orientation='vertical',
                    fraction=0.01,
                    pad=-0.15)
cbar.set_label("Decoding accuracy")

plt.tight_layout()
plt.show()

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

fig,ax = plt.subplots(1,1, figsize=(2,2))
# create dummy invisible image
# (use the colormap you want to have on the colorbar)
img = plt.imshow(np.array([[0.3,1]]), cmap="magma")
img.set_visible(False)
ax.set_axis_off()

plt.colorbar(orientation="vertical")
plt.tight_layout()
SAVE_DIR = './../../../buckets/manifold-dynamics/'
out_path = os.path.join(SAVE_DIR, f'svm_colorbar.png')
plt.savefig(out_path, dpi=300, transparent=True, bbox_inches='tight')

plt.show()

# # add any other things you want to the figure.
# plt.plot(np.random.rand(30))
